In [1]:
def performances(list_scores, list_targets, nCV):
    total_cm=[]
    total_acc =[]
    total_sen =[]
    total_spec =[]
    total_MCC =[]
    total_AUC =[]
    total_ROC =[]
    total_PR =[]
    total_kappa =[]
    total_auroc=[]
    total_bal_acc=[]
    total_y_pred =[]
    total_y_true =[]
    
    
    for j in range(nCV):
        y_pred = list_scores[j]
        y_true = list_targets[j]
        fpr, tpr, thresh_roc = roc_curve(y_true, y_pred, pos_label=1)
        prec, rec, thresh_pr = precision_recall_curve(y_true, y_pred, pos_label=1)
        total_auroc.append(roc_auc_score(y_true, y_pred))
        for i in range(len(y_pred)):
           if y_pred[i] <0.5 :
               y_pred[i] = 0
           else :
               y_pred[i] = 1
        # save the y_pred in every fold 
        
        total_y_pred.append(y_pred)
        total_cm.append(confusion_matrix(y_true, y_pred))
        total_MCC.append(matthews_corrcoef(y_true, y_pred))
        total_AUC.append(auc(fpr, tpr))
        total_ROC.append([fpr,tpr])
        total_PR.append([prec,rec])
        total_kappa.append(cohen_kappa_score(y_true, y_pred))
        
    for i in range(nCV):
        c_m = total_cm[i]
        total_cm_value=sum(sum(c_m))
        
        Accuracy = (c_m[0,0]+c_m[1,1])/total_cm_value
        
        Sensitivity = c_m[1,1]/(c_m[1,0]+c_m[1,1])
        
        Specificity = c_m[0,0]/(c_m[0,0]+c_m[0,1])
        
        total_acc.append(Accuracy)
        total_bal_acc.append((Sensitivity+Specificity)/2)
        total_sen.append(Sensitivity)
        total_spec.append(Specificity)
        
    for i in range (nCV): 
        total_acc[i]         =  round(total_acc[i],4)
        total_bal_acc[i]     =  round(total_bal_acc[i],4)
        total_sen[i]         =  round(total_sen[i],4)
        total_spec[i]        =  round(total_spec[i],4)
        total_MCC[i]         =  round(total_MCC[i],4)
        total_AUC[i]         =  round(total_AUC[i],4)
        total_kappa[i]       =  round(total_kappa[i],4)
        total_auroc[i]       =  round(total_auroc[i],4)
    total_performances= [total_acc,total_sen,total_spec, total_MCC,total_AUC,total_ROC, total_PR, total_kappa,total_auroc, total_bal_acc]
    return total_performances



def Get_Mean(perf):
    perf_mean=[]
    for i in range (5): 
        perf_mean.append(statistics.mean(perf[i]))
    
    perf_mean.append(statistics.mean(perf[7]))
    perf_mean.append(statistics.mean(perf[8]))
    perf_mean.append(statistics.mean(perf[9]))
    return  perf_mean # its classified by type of performance 

def Get_STD(perf):
    perf_std=[]
    for i in range (5): 
        perf_std.append(statistics.stdev(perf[i]))
    
    perf_std.append(statistics.stdev(perf[7]))
    perf_std.append(statistics.stdev(perf[8]))
    perf_std.append(statistics.stdev(perf[9]))
    
    return  perf_std # its classified by type of performance 



# visualization 

def Create_Tables(perf, model_title, data_type):
    
        
    perf_mean = Get_Mean(perf)
    perf_std = Get_STD(perf)
 
    Table_perf_m= PrettyTable(["Model Name", "Data Type", "m_ACC", "m_SN", "m_SP", "m_MCC", "m_AUC", "m_Kappa","m_AUROC","m_Bal_ACC"])
    Table_perf_e= PrettyTable(["Model Name", "Data Type", "e_ACC", "e_SN", "SP", "e_MCC", "e_AUC", "e_Kappa","e_AUROC","e_Bal_ACC"])
    
    Table_perf_m.add_row([model_title, data_type, str(round(perf_mean[0],3))
                         ,str(round(perf_mean[1],3)),str(round(perf_mean[2],3))
                         ,str(round(perf_mean[3],3)),str(round(perf_mean[4],3))
                         ,str(round(perf_mean[5],3)),str(round(perf_mean[6],3))
                         ,str(round(perf_mean[7],3))])
    
    Table_perf_e.add_row([model_title, data_type, str(round(perf_std[0],3))
                         ,str(round(perf_std[1],3)),str(round(perf_std[2],3))
                         ,str(round(perf_std[3],3)),str(round(perf_std[4],3))
                         ,str(round(perf_std[5],3)),str(round(perf_std[6],3))
                         ,str(round(perf_std[7],3))])
    print(Table_perf_m)
    print(Table_perf_e)


def Plot_Loss_Accuracy(nCV, total_list_train_acc, total_list_train_loss, total_list_val_acc, total_list_val_loss):
    
    for i in range(nCV):
        loss_values = total_list_train_loss[i]
        val_loss_values = total_list_val_loss[i]
        epochs = range(1, len(loss_values) + 1)
        plt.plot(epochs, loss_values, 'b', label='Training loss')
        plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
   
    for i in range(nCV):
    
        acc_values = total_list_train_acc[i]
        val_acc_values = total_list_val_acc[i]
        epochs = range(1, len(acc_values) + 1)
        plt.plot(epochs, acc_values, 'b', label='Training acc')
        plt.plot(epochs, val_acc_values, 'r', label='Validation acc')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()
        
def Plot_Performances(train_perf,test_perf, model_title, data_type):
   
    perf_train_mean = Get_Mean(train_perf)
    perf_train_std = Get_STD(train_perf)

    perf_test_mean = Get_Mean(test_perf)
    perf_test_std = Get_STD(test_perf)
    
    
    type_perf = ['ACC', 'SN', 'SP', 'MCC', 'AUC']
    x_indexes = np.arange(len(perf_train_mean)) #it should be 5 
    width = 0.3
    
      
    fig, axs = plt.subplots(1, 1,figsize=(7,7))
    fig.suptitle('Performances '+ model_title +" "+ data_type )
    #axs[0].figure(figsize=(7,5))
    #axs[1].figure(figsize=(7,5))
    
    axs.set_title( model_title)
    
    
    axs.bar(x_indexes-width/2, perf_train_mean,yerr =perf_train_std,width=width, color =['blue'], label = "train")
    
    axs.bar(x_indexes+width/2, perf_test_mean,yerr =perf_test_std,width=width, color =['green'], label ="test" )
    
    axs.legend(loc='lower right')
    axs.set(xlabel='perf types',ylabel ='perf in %')
    
    #axs[0].xticks(ticks =x_indexes, labels =type_perf)
    
    
   
    plt.setp(axs, xticks=x_indexes, xticklabels=type_perf)
    
    plt.show()    